In [1]:
import pyspark

In [2]:
df = spark.read.csv('*.csv', header='true')

In [3]:
df.printSchema()

root
 |-- Province_State: string (nullable = true)
 |-- Country_Region: string (nullable = true)
 |-- Last_Update: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long_: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Deaths: string (nullable = true)
 |-- Recovered: string (nullable = true)
 |-- Active: string (nullable = true)
 |-- FIPS: string (nullable = true)
 |-- Incident_Rate: string (nullable = true)
 |-- People_Tested: string (nullable = true)
 |-- People_Hospitalized: string (nullable = true)
 |-- Mortality_Rate: string (nullable = true)
 |-- UID: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- Testing_Rate: string (nullable = true)
 |-- Hospitalization_Rate: string (nullable = true)



In [4]:
df.createOrReplaceTempView("covid")

In [5]:
SQL = """
SELECT Province_State
          , CAST(SUM(Recovered) AS INT) Total_Recovered
          , CAST(SUM(People_Hospitalized) AS INT) AS Total_Hospitalized
          , CAST(SUM(Deaths) AS INT) AS Total_Deaths
FROM covid 
WHERE Province_State != "Recovered"
GROUP BY Province_State
ORDER BY Total_Hospitalized DESC
LIMIT 10
"""

df1 = spark.sql(SQL)
df1.show(10, False)

+--------------+---------------+------------------+------------+
|Province_State|Total_Recovered|Total_Hospitalized|Total_Deaths|
+--------------+---------------+------------------+------------+
|New York      |71661          |131507            |30285       |
|New Jersey    |null           |23570             |7598        |
|California    |null           |13412             |2121        |
|Illinois      |null           |11643             |2386        |
|Michigan      |1332           |11532             |4849        |
|Florida       |null           |8665              |1531        |
|Georgia       |null           |7863              |1423        |
|Massachusetts |null           |6915              |2444        |
|Pennsylvania  |null           |6657              |1837        |
|Louisiana     |null           |6195              |2737        |
+--------------+---------------+------------------+------------+



In [ ]:
SQL = """
SELECT     
    CAST(
          CAST(SUM(Deaths) * 100 AS INT ) /  
          ( SELECT SUM(Deaths) AS total FROM covid WHERE Province_State != "New York")          
          AS DECIMAL(8,2) 
        )         
    AS New_York_Deaths_Percent
FROM covid 
WHERE Province_State == "New York"
"""
df2 = spark.sql(SQL)
df2.show(10, False)